## Recap
- Instance Variables/Attributes comprise the state of an object
- Instance Methods/Functions are used to change the state of the object
- You access an instance variable x of object o by doing o.x
- Each object contains a dictionary that stores its attributes. These attributes can be accessed through the attribute `__dict__`
- The classes that a new class is derived from are called base classes, parent classes, or superclasses
- The derived class is either called the subclass or the child class
- Subclassing is when we use inheritance to derive one class from another

## Method Overriding

When class B is inherited from class A:
- You can override methods of class A in class B
- You can delegate part of the work to methods of class A by calling the methods of class A (either by `A.functionname()`, or using `super()`)
- You can add new methods to class B
- You can add new member variables to class B

__What happens to the attributes of the superclass?__ Are they inherited automatically? Not in Python (it follows from Python's approach to object construction that they will not be inherited). To make attributes "inherited", if you define an `__init__()` method, you will need to call the parent's `__init__()` method to allow it to properly attach the attributes to be inherited to the object under creation. You can either call the parent's `__init__()` method by explicitly naming the parent's type, or by using `super()`.

In [0]:
import math

class Shape:
    def __init__(self, color='black', filled=False):
        # Calling base class constructor from child class constructor
        self.__color = color
        self._filled = filled

    def get_color(self):
        return self.__color
        

    def set_color(self, color):
        self.__color = color
        

    def get_filled(self):
        return self.__filled
        

    def set_filled(self, filled):
        self.__filled = filled
        


class Rectangle(Shape):
    def __init__(self, length, breadth, color='black', filled=False):
        # Calling base class constructor from child class constructor 
        super().__init__(color, filled)
        
        self.__length = length
        self.__breadth = breadth
    
    def get_length(self):
        return self.__length
      
    def get_breadth(self):
        return self.__breadth
        
    
    def get_area(self):
        return self.__length * self.__breadth

    def get_perimeter(self):
        return 2 * (self.__length + self.__breadth)
        


class Circle(Shape):
    def __init__(self, radius, color='black', filled=False):
        super().__init__(color, filled)
        
        self.__radius = radius

    
    def get_radius(self):
        return self.__radius
      
    
    def get_area(self):
        return math.pi * self.__radius ** 2
        

    def get_perimeter(self):
        return math.pi * 2 * self.__radius
        
      
      
class MyRedCircle(Circle):
    def __init__(self, radius):
        # Calling base class constructor from child class constructor 
        super().__init__(radius)
        
        # Calling base class method from child class
        super().set_color('red')

## When to Use Inheritance?
Subclassing is a way of expressing that a subclass instance can be used wherever a superclass instance can be used – without breaking the code.

If substituting an object of type Base with an object of type Derived may break some code (including code that will be developed in the future) then one should NOT derive Derived from Base, while in the opposite case it is OK to derive Derived from Base.

This is called Liskov's substitution principle (LSP), and is named of Barbara Liskov, a pioneer of object oriented programming. In particular, it is BAD to use subclassing when the subclass restricts the superclass in some way.

For example, consider a cardgame where we would like to have a Deck class that represents an ordered list of cards. Deck should support removing a card from the top. It is tempting to derive Deck from list. However, this blows up the interface of Deck, adding methods, which are not meant to be used by the Deck class. In this case, Deck should not inherit from list, but rather should have an object of type as a member. This is called composition. Composition is almost always the preferred way of reusing useful functionality from one class in another.

### Replacing inheritance with composition
Sometimes we can replace inheritance with composition and achieve a similar result – this approach is sometimes considered preferable.

In [0]:
class Student:
    def __init__(self):
        self.classes = []

    def enrol(self, course):
        self.classes.append(course)


class Person:
    def __init__(self, name, surname, student=None):
        self.name = name
        self.surname = surname

        self.student = student


person = Person("Jane", "Smith", Student())
person.student.enrol("CMPUT275")

__Exercise__: Rewrite the `Person` class, adding a new role, i.e., teacher, to a person in its `__init__` method. Define the `Teacher` class and implement a method called `assign_teaching()`. This method should raise an appropriate error message if the delegation cannot be performed because the corresponding attribute has not been set.

In [4]:
class Student:
    def __init__(self):
        self.classes = []

    def enrol(self, course):
        self.classes.append(course)
      
      
class Teacher:
    def __init__(self):
        self.courses_taught = []

    def assign_teaching(self, course):
        self.courses_taught.append(course)


class Person:
    def __init__(self, name, surname, student=None, teacher=None):
        self.name = name
        self.surname = surname
        
        self.student = student
        self.teacher = teacher
    
    def __getattr__(self, name):
        if name == 'teacher' and self.name is not None:
            return self.name
        else:
            raise AttributeError


person = Person("John", "Smith", Student())
person.student.enrol("CMPUT275")
person.teacher.assign_teaching("CMPUT274")

AttributeError: ignored

## Multiple Inheritance
A class can inherit from multiple classes at the same time

In [0]:
class ParentClass_1:
    def explore(self):
        print("explore() method called")


class ParentClass_2:
    def search(self):
        print("search() method called")


class ParentClass_3:
    def discover(self):
        print("discover() method called")


class ChildClass(ParentClass_1, ParentClass_2, ParentClass_3):
    def test(self):
        print("test() method called")

In [0]:
obj = ChildClass()

obj.explore()
obj.search()
obj.discover()
obj.test()

explore() method called
search() method called
discover() method called
test() method called


## Method Resolution Order
We can derive a class from multiple classes. What changes is that in the attribute lookup strategy one has to account for that a single class may have multiple parents. Python uses the so-called __C3 strategy__, which is based on the following principles:
- Children's namespaces are always looked up first before parents' namespaces
- Attribute lookup respects the declaration order of the baseclasses. 
If in the definition A, the base classes are B1, B2, B3 in this order then in the lookup B1's namespace is checked before checking B2's namespace, which is checked before B3's namespace.
- Attribute lookup works in a depth first manner: Checking parent's is given priority as long as this does not contradict the first two rules mentioned.

The actual Method Resolution Order (MRO) is computed at the time the object holding the class definition is created (it happens in the `__new__` method) and is stored in the class attribute `__mro__`. Thus, one can print in which order the attributes will be looked up when starting from class A by using `print(A.__mro__)`.
If you plan to use multiple inheritance (this should be rare), you must lookup how `super()` works. You must use `super()` to properly initialize member variables in the presence of multiple inheritance.

In [0]:
print(ChildClass.__mro__)

(<class '__main__.ChildClass'>, <class '__main__.ParentClass_1'>, <class '__main__.ParentClass_2'>, <class '__main__.ParentClass_3'>, <class 'object'>)


In [0]:
class A:
    def __init__(self):
        print('A: before init')
        super().__init__()
        print('A: after init')

class B(A):
    def __init__(self):
        print('B: before init')
        super().__init__()
        print('B: after init')
        
class C:
    def __init__(self):
        print('C: before init')
        super().__init__()
        print('C: after init')
        
class D(C, B):
    def __init__(self):
        print('D: before init')
        super().__init__()
        print('D: after init')

In [8]:
print(D.__mro__)

(<class '__main__.D'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)


In [9]:
# the lookup is triggered by super() will proceed through the entire superclass 
# list of B, before working through the same for C.
d = D()

D: before init
C: before init
B: before init
A: before init
A: after init
B: after init
C: after init
D: after init


In [0]:
print(D.__mro__)

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.A'>, <class '__main__.C'>, <class 'object'>)


In [12]:
class Grandparent:
    def __init__(self):
        self.name = "Grandparent"
    
    def foo(self):
        print(self.name, "method called")
        
    def bar(self):
        print("Grandparent method called")


class Parent1(Grandparent):
    def __init__(self):
        self.name = "Parent1"
        
    def foo(self):
        print(self.name, "method called")
        
    def bar(self):
        print(self.name, "method called")

        
class Parent2(Grandparent):
    def __init__(self):
        self.name = "Parent2"


class Child1(Parent1, Parent2):
    def __init__(self):
        self.name = "Child1"
        

      
obj1 = Child1()

obj1.foo()
obj1.bar()

Child1 method called
Child1  method called


In [13]:
class Child2(Parent2, Parent1):
    def __init__(self):
        self.name = "Child2"

obj2 = Child2()

obj2.foo()
obj2.bar()

Child2 method called
Child2  method called


In [14]:
print(Child1.__mro__)
print(Child2.__mro__)

(<class '__main__.Child1'>, <class '__main__.Parent1'>, <class '__main__.Parent2'>, <class '__main__.Grandparent'>, <class 'object'>)
(<class '__main__.Child2'>, <class '__main__.Parent2'>, <class '__main__.Parent1'>, <class '__main__.Grandparent'>, <class 'object'>)


In [0]:
class Grandchild(Child1, Child2):
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases Parent1, Parent2

### When multiple inheritance should be used?
Use multiple inheritance when the different base classes provide distinct “services” (usually you would name such classes using adverbs, the base classes would also be called "mixin"s as they are expected to be mixed). Also, this works best when their functions have distinct names.

Multiple inheritance can cause a lot of ambiguity and confusion. Therefore, we should minimize its use.

## Polymorphism and Method Overriding 
Sometimes an object comes in many forms but all forms do share the same logic. Thus, we run this logic using the same method. This idea is called Polymorphism.

To override a method in the base class, the subclass needs to define a method with the same signature.


In [0]:
class A:
    def explore(self):
        print("explore() method from class A")

class B(A):
    def explore(self):
        # calling the parent class explore() method
        # super().explore()  
        print("explore() method from class B")


def runExplore(obj):
    obj.explore()

    
b_obj = B()
a_obj = A()

runExplore(b_obj)
runExplore(a_obj)

print(isinstance(b_obj, A))
print(isinstance(b_obj, B))

explore() method from class B
explore() method from class A
True
True


### Polymorphism with Abstract Class

---

![](https://pythonspot-9329.kxcdn.com/wp-content/uploads/2016/03/polymorphism.png)


In [17]:
class Document:
  def __init__(self, name):    
    self.name = name
    
  def show(self):
    raise NotImplementedError("Subclass must implement abstract method")
    
class Pdf(Document):
  def show(self):
    return 'Show pdf contents!'
  
class Word(Document):
  def show(self):
    return 'Show word contents!'

documents = [Pdf('Document1'), Word('Document2'), Word('Document3')]
 
for document in documents:
    print(document.name + ': ' + document.show())

Document1: Show pdf contents!
Document2: Show word contents!
Document3: Show word contents!


In [42]:
"""A module with talking animals."""

class Animal(object):
    def __init__(self, name):
        self.name = name

    def speak(self):
        print(self.name, 'says', self.sound())
        
    def sound(self):
        raise NotImplementedError("Subclass must implement abstract method")

class Cow(Animal):
    def __init__(self, name):
        super(Cow, self).__init__(name)

    def sound(self):
        return 'moo'

class Horse(Animal):
    def __init__(self, name):
        super(Horse, self).__init__(name)

    def sound(self):
        return 'neigh'

class Sheep(Animal):
    def __init__(self, name):
        super(Sheep, self).__init__(name)

    def sound(self):
        return 'baaaaa'

if __name__ == '__main__':
    animals = [Horse('Little Horse'), Cow('Little Cow'), Sheep('Little Lamb')]

    for animal in animals:
        animal.speak()

Little Horse says neigh
Little Cow says moo
Little Lamb says baaaaa


## Inner Class
An inner class or nested class is a class that is defined within the body of another class. Inner classes are rarely used in Python!

In [49]:
class Human:
    def __init__(self):
        self.name = 'C3PO'
        self.mouth = self.Mouth()
        self.brain = self.Brain()
        
    def talk(self):
        return self.mouth.talk()
        
        
    def think(self):
        return self.brain.think()

        
    class Mouth:
        # here you cannot see any members of class Human!
        def talk(self):
            return 'talking...'
        
    class Brain:
        def think(self):
            return 'thinking...'

          
h = Human()
print(type(h))
print(type(h.mouth))
print(type(h.brain))

m = h.Mouth()
print(type(m))


print(h.name)
print(h.talk())
print(h.think())

<class '__main__.Human'>
<class '__main__.Human.Mouth'>
<class '__main__.Human.Brain'>
C3PO
talking...
thinking...
<class '__main__.Human.Mouth'>


## Design Patterns
Design patterns are proven solutions to common problems in a specific context. 
The Design Patterns book, known as the Gang of Four (GoF), 
discusses 23 different patterns, classified under three purposes: creational patterns, structural patterns, and behavioral patterns. Creational patterns address object instantiation issues. Structural patterns concentrate on object composition and their relations in the runtime object structures. Behavioral patterns focus on the internal dynamics and object interaction in the system.

![](https://images-na.ssl-images-amazon.com/images/I/51kuc0iWoKL._SX326_BO1,204,203,200_.jpg)

### Creational Patterns
These design patterns concern creating objects.

#### Factory Method

We may not always know what kind of objects we want to create in advance.
Some objects can be created only at execution time after a user requests so.
<br>

__Example(s)__: 
- A user may click on a certain button that creates an object.
- A user may create documents of different types.

<br>
__Implementation__:
Create a function, namely the factory, that takes an input __string__ and outputs an __object__. 
Factory is a _static_ method, i.e., it is bound to a class rather than its object; hence, it doesn't require a class instance creation.

In [41]:
class Document:
    def factory(type):
        if type == "PdfDocument": 
            return PdfDoc()
        if type == "WordDocument": 
            return WordDoc()
        raise AssertionError("Bad document creation: " + type)
        
    # staticmethod() converts the given function to a static method
    # you can use the Python decorator @staticmethod 
    # which will be covered later    
    factory = staticmethod(factory)


# Subclasses of Document
class PdfDoc(Document):
    def show(self): 
        print("Showing a PDF...")

        
class WordDoc(Document):
    def show(self): 
        print("Showing a MS Word...")

        
if __name__ == "__main__":  
    # Create object using the factory method.
    # Calling a static method does not require creating an instance
    firstdoc = Document.factory("PdfDocument")
    firstdoc.show()

    seconddoc = Document.factory("WordDocument")
    seconddoc.show()

    thirddoc = Document.factory("RtfDocument")

Showing a PDF...
Showing a MS Word...


#### Singleton
The Singleton pattern is used when we want to guarantee that only one instance of a given class exists during runtime.
<br>

__Examples__:
- Controlling concurrent access to a shared resource
- Managing database connections
- Printer spooler
- Logging class

__Implementation__: Modify the constructor to store the first instance of a class in a class attribute. 
Return the same attribute every time that the constructor is called.

In [21]:
class Logger:
    def __new__(cls):
        if not hasattr(cls, '_logger'):
            cls._logger = super().__new__(cls)
        return cls._logger


if __name__ == '__main__':
    objA = Logger()
    objB = Logger()
    print('objA id: {0}'.format(id(objA)))
    print('objB id: {0}'.format(id(objB)))

objA id: 140661925795264
objB id: 140661925795264


__Exercise__: Create a Singleton class, `OnlyK`, which can create to `K` instances.
Assume the objects are database connections and you only have a license to use a fixed quantity of these at any one time.

In [23]:
K = 3

class OnlyK(object):
    '''Singleton class which manages a fixed number of its own objects'''
    __instance_count = 0
    
    def __new__(cls):
        if cls.__instance_count < K:
            cls.__instance_count += 1
            cls.__last_instance = super().__new__(cls)
        return cls.__last_instance
  

# Test your implementation
if __name__ == '__main__':
    first = OnlyK()
    print('obj id: {0}'.format(id(first)))
    second = OnlyK()
    print('obj id: {0}'.format(id(second)))
    third = OnlyK()
    print('obj id: {0}'.format(id(third)))
    fourth = OnlyK()
    print('obj id: {0}'.format(id(fourth)))

obj id: 139760682379416
obj id: 139760682377792
obj id: 139760682456120
obj id: 139760682456120
